### final

In [2]:
## datadam_commercial_population_score

## 확인사항
## 1. KT데이터 업데이트 여부
## 2. 건축물대장 업데이트 여부
## 3. 실거래가 업데이트 여부

# module
import psycopg2
import pandas as pd

# 기준월
date = '202211'
# dates = pd.date_range('2022-01',periods=10,freq='M').strftime("%Y%m")

# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# 시군구코드
aws_cursor.execute(
"""
select distinct substring(pnu, 1, 5) as sig_cd
from m1.pnu
order by 1
"""
)
aws_conn.commit()
rows = aws_cursor.fetchall()
df = pd.DataFrame(rows, columns = ['sig_cd'])
lawd_cd = list(df['sig_cd'])


# INSERT
for i in lawd_cd:
    aws_cursor.execute(
    """
    insert into m2.datadam_commercial_population_score (
    with date as (
        select '""" + date + """'::text as base_ym
    )
    select (select base_ym from date) as base_ym
         , A.pnu
         , substring(A.pnu, 1, 8) as emd_cd
         , case when
                round(
                        (
                            nvl((avg(B.f_tot) - avg(C.f_tot_avg)) / avg(C.f_tot_std) * 5 + 75, 75) +
                            nvl((avg(D.deal_amount_per_area) - avg(E.deal_amount_avg)) / avg(E.deal_amount_std) * 5 + 75, 75)
                        ) / 2, 2
                    ) > 100 then 100
                else
                round(
                        (
                            nvl((avg(B.f_tot) - avg(C.f_tot_avg)) / avg(C.f_tot_std) * 5 + 75, 75) +
                            nvl((avg(D.deal_amount_per_area) - avg(E.deal_amount_avg)) / avg(E.deal_amount_std) * 5 + 75, 75)
                        ) / 2, 2
                    ) end as score
         , sysdate::date as create_at
         , sysdate::date as update_at
         , 'du.Park' as work_user
    from (
        -- 모집단 가져오기 : KT매출 & PNU존재 & 건물존재
        select T1.pnu
             , ST_Centroid(T2.poly::geometry) as poly
        from (
            -- KT매출 존재
            select distinct substring(bd_mgt_sn, 1, 19) as pnu
            from m1.kt_bldg_sales
            where 1=1
                and base_ym = (select base_ym from date) -- 기준월
                and substring(bd_mgt_sn, 1, 5) = '""" + str(i) + """' -- 시군구
        ) T1
        -- PNU 존재
        join m2.cremao_land T2
            on T1.pnu = T2.pnu
        -- 건물 존재
        join (
            select distinct sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu
            from m1.bld_rgst
            where base_ym = (select base_ym from date) -- 기준월
        ) T3
            on T1.pnu = T3.pnu
    ) A
    left join (
        select substring(bd_mgt_sn, 1, 19) as pnu
             , avg(f_tot) as f_tot
        from m1.kt_bldg_xy_info
        where 1=1
            and base_ym = (select base_ym from date) -- 기준월
            and substring(bd_mgt_sn, 1, 5) = '""" + str(i) + """' -- 시군구
        group by 1
    ) B
        on A.pnu = B.pnu
    left join (
        select avg(f_tot) as f_tot_avg
             , stddev(f_tot) as f_tot_std 
        from m1.kt_bldg_xy_info
        where 1=1
            and base_ym = (select base_ym from date) -- 기준월
            and substring(bd_mgt_sn, 1, 5) = '""" + str(i) + """' -- 시군구
    ) C
        on 1=1
    left join (
        select ST_Centroid(T2.poly::geometry) as poly
             , round(1.0 * T1.deal_amount * 10000 / T1.building_area, 0) as deal_amount_per_area
        from m2.cremao_building_trade_case T1
        left join m2.cremao_land T2
            on T1.pnu = T2.pnu
        where 1=1
            and T1.contract_year_month between (select dateadd(month, -59, to_date(base_ym||'01', 'yyyymmdd'))::date from date) and (select last_day(to_date(base_ym||'01', 'yyyymmdd')) from date) -- 최근 3년
            and substring(T1.pnu, 1, 5) = '""" + str(i) + """' -- 시군구
    ) D
        on ST_DistanceSphere(A.poly, D.poly) < 150 -- 반경 150m PNU
    left join (
        select round(avg(1.0 * deal_amount * 10000 / building_area), 0) as deal_amount_avg
             , round(stddev(1.0 * deal_amount * 10000 / building_area), 0) as deal_amount_std
        from m2.cremao_building_trade_case
        where 1=1
            and contract_year_month between (select dateadd(month, -59, to_date(base_ym||'01', 'yyyymmdd'))::date from date) and (select last_day(to_date(base_ym||'01', 'yyyymmdd')) from date) -- 최근 3년
            and substring(pnu, 1, 5) = '""" + str(i) + """' -- 시군구
    ) E
        on 1=1
    group by 1, 2, 3
    );
    """
    )
    aws_conn.commit()
    print(i, "Done")

11110 Done
11140 Done
11170 Done
11200 Done
11215 Done
11230 Done
11260 Done
11290 Done
11305 Done
11320 Done
11350 Done
11380 Done
11410 Done
11440 Done
11470 Done
11500 Done
11530 Done
11545 Done
11560 Done
11590 Done
11620 Done
11650 Done
11680 Done
11710 Done
11740 Done
26110 Done
26140 Done
26170 Done
26200 Done
26230 Done
26260 Done
26290 Done
26320 Done
26350 Done
26380 Done
26410 Done
26440 Done
26470 Done
26500 Done
26530 Done
26710 Done
27110 Done
27140 Done
27170 Done
27200 Done
27230 Done
27260 Done
27290 Done
27710 Done
28110 Done
28140 Done
28177 Done
28185 Done
28200 Done
28237 Done
28245 Done
28260 Done
28710 Done
28720 Done
29000 Done
29110 Done
29140 Done
29155 Done
29170 Done
29200 Done
30110 Done
30140 Done
30170 Done
30200 Done
30230 Done
31110 Done
31140 Done
31170 Done
31200 Done
31710 Done
36110 Done
41111 Done
41113 Done
41115 Done
41117 Done
41131 Done
41133 Done
41135 Done
41150 Done
41171 Done
41173 Done
41190 Done
41195 Done
41197 Done
41199 Done
41210 Done

###### ver1 (법정동 단위)

In [31]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.6f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# pnu별 상업용 부동산 수
sql = '''
select sigungu_cd||bjdong_cd as pnu, count(mgm_bldrgst_pk) as total_count
  from m1.bld_rgst
 where main_purps_cd_nm not in 
    (
    select distinct main_purps_cd_nm
    from m1.bld_rgst
    where main_purps_cd_nm like '%주택%'
    )
group by sigungu_cd||bjdong_cd
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgst = pd.DataFrame(rows)
df_bld_rgst.columns = ['emd_cd','total_count']

# 법정동별 유동인구 
sql = '''
    select substring (pnu, 0, 11), sum(f_tot)
      from m2.cremao_land_floating_population
     group by substring (pnu, 0, 11)
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_f_tot = pd.DataFrame(rows)
df_f_tot.columns = ['emd_cd','f_tot']
df_f_tot = df_f_tot[~df_f_tot['f_tot'].isnull()].reset_index(drop=True)

# 데이터 프레임 병합
df = pd.merge(df_f_tot, df_bld_rgst, on='emd_cd', how='left')

# 인구가 0명, null인 지역 제외
df = df[~df['total_count'].isnull()]

# 상가면적 / 거주인구
df['population_per_commercial'] = (df['f_tot'] / df['total_count'])
df = df[['emd_cd','population_per_commercial']]
df['base_ym'] = datetime.today().strftime("%Y%m")


### 2. DB INSERT
import datetime

table = 'm2.datadam_population_per_commercial'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
     # drop & create
    cursor.execute(
    """
    drop table if exists m2.datadam_population_per_commercial;
    create table m2.datadam_population_per_commercial (    
    emd_cd varchar(10)
    , population_per_commercial double precision
    , base_ym varchar(6)
    )
    distkey(emd_cd)
    compound sortkey(base_ym);
    commit;
    """
    )
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-10-30 19:26:35.944487  :  0
2022-10-30 19:26:38.039882  :  10000
17296


In [30]:
df

,emd_cd,population_per_commercial,base_ym
0,1111010100,71464.562500,202210
1,1111010200,132057.057143,202210
2,1111010300,25740.906977,202210
3,1111010600,362829.345455,202210
4,1111011000,664140.230769,202210
...,...,...,...
18597,4889039030,4.606061,202210
18598,4889041026,41.153846,202210
18599,4889041030,11.597015,202210
18600,4889042022,2.032258,202210


In [32]:
df.describe()

,population_per_commercial
count,17296.000000
mean,45176.698724
std,153540.963842
min,0.058480
25%,49.351316
50%,321.275980
75%,3864.073727
max,3469255.572881


###### ver2 (pnu 단위)

In [17]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.6f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# pnu별 상업용 부동산 수
sql = '''
select count(mgm_bldrgst_pk), sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji as pnu
  from m1.bld_rgst
 where main_purps_cd_nm not in 
	(
	select distinct main_purps_cd_nm
	from m1.bld_rgst
	where main_purps_cd_nm like '%주택%'
	)
group by sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgst = pd.DataFrame(rows)
df_bld_rgst.columns = ['bld_count','pnu']

# pnu별 주용도명
sql = '''
select main_purps_cd_nm, sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji as pnu
  from m1.bld_rgst
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_principal_use = pd.DataFrame(rows)
df_bld_principal_use.columns = ['principal_use','pnu']

# pnu별 유동인구 
sql = 'select pnu, f_tot from m2.cremao_land_floating_population'
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_f_tot = pd.DataFrame(rows)
df_f_tot.columns = ['pnu','f_tot']


# 데이터 프레임 병합
df = pd.merge(df_f_tot, df_bld_rgst, on='pnu', how='left')

# 인구가 0명, null인 지역 제외
df = df[~df['bld_count'].isnull()]

# 상업용 부동산 수 / 유동인구
df['population_per_commercial'] = (df['f_tot'] / df['bld_count'])

# 주용도코드 붙이기
df = pd.merge(df, df_bld_principal_use, on='pnu', how='left')
df = df.drop_duplicates('pnu').reset_index(drop=True)

# inf 값 제거후 컬럼 정리
df = df[~(df['population_per_commercial']==np.inf)]
df = df[['pnu', 'principal_use', 'population_per_commercial']]
df['base_ym'] = datetime.today().strftime("%Y%m")


### 2. DB INSERT
import datetime

table = 'm2.datadam_population_per_commercial'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
     # drop & create
    cursor.execute(
    """
    drop table if exists m2.datadam_population_per_commercial;
    create table m2.datadam_population_per_commercial (    
    pnu varchar(19)
    , principal_use varchar(200)
    , population_per_commercial double precision
    , base_ym varchar(6)
    )
    distkey(pnu)
    compound sortkey(principal_use, base_ym);
    commit;
    """
    )
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-11-01 10:29:47.219399  :  0
2022-11-01 10:29:50.140698  :  10000
2022-11-01 10:29:53.516130  :  20000
2022-11-01 10:29:56.655727  :  30000
2022-11-01 10:30:05.693395  :  40000
2022-11-01 10:30:12.183393  :  50000
2022-11-01 10:30:15.093622  :  60000
2022-11-01 10:30:19.504118  :  70000
2022-11-01 10:30:22.047369  :  80000
2022-11-01 10:30:27.890195  :  90000
2022-11-01 10:30:33.740693  :  100000
2022-11-01 10:30:37.349954  :  110000
2022-11-01 10:30:40.307199  :  120000
2022-11-01 10:30:43.708485  :  130000
2022-11-01 10:30:49.001870  :  140000
2022-11-01 10:30:54.940209  :  150000
2022-11-01 10:30:58.136175  :  160000
2022-11-01 10:31:03.054055  :  170000
2022-11-01 10:31:05.827197  :  180000
2022-11-01 10:31:09.834254  :  190000
2022-11-01 10:31:12.323241  :  200000
2022-11-01 10:31:15.321996  :  210000
2022-11-01 10:31:19.141912  :  220000
2022-11-01 10:31:21.767593  :  230000
2022-11-01 10:31:25.940078  :  240000
2022-11-01 10:31:28.420667  :  250000
2022-11-01 10:31:32.117601

In [15]:
df

,pnu,principal_use,population_per_commercial,base_ym
0,1111010800100100003,제2종근린생활시설,176465.666667,202210
1,1111010800100210001,제1종근린생활시설,176465.666667,202210
2,1111010800100310002,근린생활시설,81002.000000,202210
3,1111010800100310025,제1종근린생활시설,176465.666667,202210
4,1111010800100370002,제1종근린생활시설,81002.000000,202210
...,...,...,...,...
732242,5013031029103510000,제2종근린생활시설,911.600000,202210
732243,5013031029200880001,종교시설,5432.666667,202210
732244,5013031030103800007,숙박시설,709.333333,202210
732245,5013031030104270000,종교시설,354.666667,202210


In [16]:
df.describe()

,population_per_commercial
count,732247.000000
mean,78341.192311
std,120098.742693
min,0.000000
25%,6888.733333
50%,41872.666667
75%,100175.583333
max,5101176.000000


###### ver3 (final)

In [ ]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.6f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# pnu별 상업용 부동산 수
sql = '''
select count(mgm_bldrgst_pk), sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji as pnu
  from m1.bld_rgst
 where main_purps_cd_nm not in 
	(
	select distinct main_purps_cd_nm
	from m1.bld_rgst
	where main_purps_cd_nm like '%주택%'
	)
  and base_ym = '202209'
group by sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgst = pd.DataFrame(rows)
df_bld_rgst.columns = ['bld_count','pnu']
df_bld_rgst['bun'] = df_bld_rgst['pnu'].str[:15] # 15: 본번까지 구분해서 병합
df_bld_rgst['emd_cd'] = df_bld_rgst['pnu'].str[:8] + '00'

# pnu별 유동인구 
sql = '''
select substring(bd_mgt_sn, 0, 20) as pnu, avg(f_tot)
  from m1.kt_bldg_xy_info
 where base_ym = '202209'
 group by substring(bd_mgt_sn, 0, 20)
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_f_tot = pd.DataFrame(rows)
df_f_tot.columns = ['pnu_x','f_tot']
df_f_tot['bun'] = df_f_tot['pnu_x'].str[:15]

# 데이터 프레임 병합
df = pd.merge(df_bld_rgst, df_f_tot, on='bun', how='left')

# 인구가 0명, null인 지역 제외
df = df[~df['pnu_x'].isnull()]

# 유동인구 / 상업용 부동산 수 
df['population_per_commercial'] = (df['f_tot'] / df['bld_count'])

# inf 값 제거후 컬럼 정리
df = df[~(df['population_per_commercial']==np.inf)]
df = df[['pnu','emd_cd','population_per_commercial']]
df['create_at'] = datetime.today().strftime("%Y-%m-%d")
df['update_at'] = datetime.today().strftime("%Y-%m-%d")
df['work_user'] = 'du.Park'


### 2. DB INSERT
import datetime

table = 'm2.datadam_population_per_commercial'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

###### ver4

In [ ]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.6f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# pnu별 상업용 부동산 수
sql = '''
select count(mgm_bldrgst_pk), sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji as pnu
  from m1.bld_rgst
 where main_purps_cd_nm not in 
	(
	select distinct main_purps_cd_nm
	from m1.bld_rgst
	where main_purps_cd_nm like '%주택%'
	)
  and base_ym = '202209'
group by sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgst = pd.DataFrame(rows)
df_bld_rgst.columns = ['bld_count','pnu']
df_bld_rgst['bun'] = df_bld_rgst['pnu'].str[:15] # 15: 본번까지 구분해서 병합
df_bld_rgst['emd_cd'] = df_bld_rgst['pnu'].str[:8] + '00'

# pnu별 유동인구 
sql = '''
select substring(bd_mgt_sn, 0, 20) as pnu, avg(f_tot)
  from m1.kt_bldg_xy_info
 where base_ym = '202209'
 group by substring(bd_mgt_sn, 0, 20)
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_f_tot = pd.DataFrame(rows)
df_f_tot.columns = ['pnu_x','f_tot']
df_f_tot['bun'] = df_f_tot['pnu_x'].str[:15]

# 데이터 프레임 병합
df = pd.merge(df_bld_rgst, df_f_tot, on='bun', how='left')

# 인구가 0명, null인 지역 제외
df = df[~df['pnu_x'].isnull()]

# 유동인구 / 상업용 부동산 수 
df['population_per_commercial'] = (df['f_tot'] / df['bld_count'])

# inf 값 제거후 컬럼 정리
df = df[~(df['population_per_commercial']==np.inf)]
df = df[['pnu','emd_cd','population_per_commercial']]
df['create_at'] = datetime.today().strftime("%Y-%m-%d")
df['update_at'] = datetime.today().strftime("%Y-%m-%d")
df['work_user'] = 'du.Park'


### 2. DB INSERT
import datetime

table = 'm2.datadam_population_per_commercial'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

In [19]:
df

,pnu,emd_cd,population_per_commercial,create_at,update_at,work_user
0,1111010100100030100,1111010100,3988060.000000,2022-11-07,2022-11-07,du.Park
1,1111010100100060003,1111010100,1444137.000000,2022-11-07,2022-11-07,du.Park
2,1111010100100150005,1111010100,2198691.000000,2022-11-07,2022-11-07,du.Park
3,1111010100100150005,1111010100,2198691.000000,2022-11-07,2022-11-07,du.Park
4,1111010100100150005,1111010100,2198691.000000,2022-11-07,2022-11-07,du.Park
...,...,...,...,...,...,...
9734808,5013032026115240006,5013032000,97462.000000,2022-11-07,2022-11-07,du.Park
9734809,5013032026115240006,5013032000,97462.000000,2022-11-07,2022-11-07,du.Park
9734810,5013032026115240011,5013032000,97462.000000,2022-11-07,2022-11-07,du.Park
9734811,5013032026115240011,5013032000,97462.000000,2022-11-07,2022-11-07,du.Park


### RobustScaler

In [27]:
from sklearn.preprocessing import RobustScaler

# 변형 객체 생성
robust_scaler = RobustScaler()

# 훈련데이터의 모수 분포 저장
robust_scaler.fit(df)

# 훈련 데이터 스케일링
df_scaled = robust_scaler.transform(df)

In [28]:
df_scaled

array([[-7.65643687, 18.64966277,  0.        ],
       [-7.65643665, 34.53351698,  0.        ],
       [-7.65643642,  6.66355982,  0.        ],
       ...,
       [ 0.83376818, -0.08117995,  0.        ],
       [ 0.83377041, -0.08368727,  0.        ],
       [ 0.83377268, -0.07886248,  0.        ]])